In [2]:
import pandas as pd 
import numpy as np
import calendar
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv("/Users/rasmusblirupjensen/Downloads/Parking_Violations_Issued_-_Fiscal_Year_2019.csv")
codes = pd.read_excel("/Users/rasmusblirupjensen/Downloads/ParkingViolationCodes_Nov_2018.xlsx")
df = df.sample(frac=0.50)

In [8]:
df = df.rename(columns={"Violation Code": "Code"})
df['Date']=pd.to_datetime(df["Issue Date"], format='%m/%d/%Y', errors='coerce')
df["Day Name"] = df["Date"].apply(lambda x: x.weekday_name)
len(df)



/Users/rasmusblirupjensen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


5733753

In [9]:
#Sheet with the violation descriptions
codes = codes.rename(columns={"VIOLATION CODE": "Code"})
codes


,Code,VIOLATION DESCRIPTION,Manhattan 96th St. & below\n(Fine Amount $),All Other Areas\n(Fine Amount $)
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,115,115
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,65,65
93,96,RAILROAD CROSSING,95,95
94,97,VACANT LOT,65,45
95,98,OBSTRUCTING DRIVEWAY,95,95


In [10]:
#Adding the violation discription to the dataframe
result =  df.merge(codes, on='Code', how='left')
result.head()


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,Date,Day Name,VIOLATION DESCRIPTION,Manhattan 96th St. & below\n(Fine Amount $),All Other Areas\n(Fine Amount $)
0,4648740518,17387SH,NY,SCL,10/11/2018,36,SUBN,COLLI,V,0,...,NaN,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN,2018-10-11,Thursday,PHTO SCHOOL ZN SPEED VIOLATION,50.0,50.0
1,8572430246,96328MB,NY,COM,08/31/2018,14,VAN,CHEVR,T,37510,...,66,NaN,NaN,NaN,NaN,2018-08-31,Friday,NO STANDING-DAY/TIME LIMITS,115.0,115.0
2,8648976674,AS697F,NY,PAS,06/30/2018,38,4DSD,HONDA,T,5280,...,26,NaN,NaN,NaN,NaN,2018-06-30,Saturday,FAIL TO DSPLY MUNI METER RECPT,65.0,35.0
3,8690712719,HRA6725,NY,PAS,01/08/2019,53,SUBN,TOYOT,T,29290,...,50-A,NaN,NaN,NaN,NaN,2019-01-08,Tuesday,SAFETY ZONE,115.0,115.0
4,8653632426,52110MH,NY,COM,08/30/2018,20,VAN,FORD,T,49020,...,03,NaN,NaN,NaN,NaN,2018-08-30,Thursday,NO PARKING-DAY/TIME LIMITS,65.0,60.0


In [11]:
#Top 5 most violtated parkings crimes
topfines = result["VIOLATION DESCRIPTION"].value_counts()[:5]
topfines

NO PARKING-STREET CLEANING        901896
FAIL TO DSPLY MUNI METER RECPT    583070
PHTO SCHOOL ZN SPEED VIOLATION    549483
NO STANDING-DAY/TIME LIMITS       505774
NO PARKING-DAY/TIME LIMITS        397556
Name: VIOLATION DESCRIPTION, dtype: int64

In [12]:
fines = topfines.index.to_list()

In [13]:
result_new = result.loc[result['VIOLATION DESCRIPTION'].isin(fines)]

In [14]:
#Top 5 crimes % size of total set 
print(len(result_new)/len(df)*100 ,"%")

51.23658099677472 %


In [16]:
#Length and number of variables in the dataset 
print("Number of rows: {}  \nNumber of Variables: {}" .format(result_new.shape[0], str(result_new.shape[1])))

Number of rows: 2937779  
Number of Variables: 48


#Data Properties 
In the dataset, there is geolocation attributes, such as strees names, house numbers, states codes etc. Next there is description of the different vehicles. There is an issue date which tells us when the violation has happened, and a timestamp for the violation. 

In [24]:
result_new.columns
result_new['Violation Time']

0          0918A
1          0255P
2          1139A
4          0146P
5          0208P
           ...  
5733742    0017A
5733745    0155P
5733747    1057A
5733749    0104P
5733752    0906A
Name: Violation Time, Length: 2937779, dtype: object

In [73]:
#result.shape
#result["Adress"] = result["Street Name"] + " " + str(result["House Number"])

In [17]:
#result_new.to_csv("PARKING2019.csv")

In [75]:
#result.columns

In [25]:
a= "0918A"